# Reading and preparing data

In [170]:
import seaborn as sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [171]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [172]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


__Categorical__: Survived, Sex, and Embarked. __Ordinal__: Pclass.
__Continous__: Age, Fare. __Discrete__: SibSp, Parch.

In [173]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [174]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [175]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [176]:
train = train.drop(['Name', 'Ticket', 'PassengerId'], axis=1)
test = test.drop(['Name', 'Ticket', 'PassengerId'], axis=1)

In [177]:
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Fill Embraked NA with S (most friquent value)

In [178]:
train['Embarked'] = train['Embarked'].fillna('S')

In [179]:
train['Embarked'] = train['Embarked'].astype('category')
train['Embarked'] = train['Embarked'].cat.codes

In [180]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,2
1,1,1,female,38.0,1,0,71.2833,C85,0
2,1,3,female,26.0,0,0,7.9250,NaN,2
3,1,1,female,35.0,1,0,53.1000,C123,2
4,0,3,male,35.0,0,0,8.0500,NaN,2


Do it for test too

In [181]:
train = train.drop(['Embarked'], axis=1)
test = test.drop(['Embarked'], axis=1)

In [182]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin
0,0,3,male,22.0,1,0,7.2500,NaN
1,1,1,female,38.0,1,0,71.2833,C85
2,1,3,female,26.0,0,0,7.9250,NaN
3,1,1,female,35.0,1,0,53.1000,C123
4,0,3,male,35.0,0,0,8.0500,NaN


Now for sex

In [183]:
train['Sex'] = train['Sex'].astype('category')
train['Sex'] = train['Sex'].cat.codes

test['Sex'] = test['Sex'].astype('category')
test['Sex'] = test['Sex'].cat.codes

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin
0,0,3,1,22.0,1,0,7.2500,NaN
1,1,1,0,38.0,1,0,71.2833,C85
2,1,3,0,26.0,0,0,7.9250,NaN
3,1,1,0,35.0,1,0,53.1000,C123
4,0,3,1,35.0,0,0,8.0500,NaN


For the first time wi will drop Cabin becasue of a lot of nans

In [184]:
train.drop("Cabin", axis=1, inplace=True)
test.drop("Cabin", axis=1, inplace=True)

In [185]:
# get average, std, and number of NaN values in titanic_df
average_age_titanic   = train["Age"].mean()
std_age_titanic       = train["Age"].std()
count_nan_age_titanic = train["Age"].isnull().sum()

# get average, std, and number of NaN values in test_df
average_age_test   = test["Age"].mean()
std_age_test       = test["Age"].std()
count_nan_age_test = test["Age"].isnull().sum()

# generate random numbers between (mean - std) & (mean + std)
rand_1 = np.random.randint(average_age_titanic - std_age_titanic, average_age_titanic + std_age_titanic, size = count_nan_age_titanic)
rand_2 = np.random.randint(average_age_test - std_age_test, average_age_test + std_age_test, size = count_nan_age_test)

train["Age"][np.isnan(train["Age"])] = rand_1
test["Age"][np.isnan(test["Age"])] = rand_2

train['Age'] = train['Age'].astype(int)
test['Age'] = test['Age'].astype(int)

In [186]:
test["Fare"].fillna(test["Fare"].median(), inplace=True)

Split targets

In [187]:
y_train = train["Survived"]

x_train = train.drop(labels = ["Survived"],axis = 1)

# Stupid classification 

Split on train and validation

In [188]:
xx_train, x_val, yy_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [192]:
parameters = {'C':[1, 10]}
linear_clf = LinearSVC()

opt_linear_clf = GridSearchCV(linear_clf, parameters, cv=5, scoring='accuracy')
opt_linear_clf.fit(x_train, y_train)

print("Best params:", opt_linear_clf.best_params_)

Best params: {'C': 1}


In [193]:
prediction = opt_linear_clf.predict(x_val)
linear_result = accuracy_score(y_val, prediction)

print('Result accuracy:', linear_result)

Result accuracy: 0.8444444444444444


In [194]:
def save_prediction(prediction, file_name):
    pd.DataFrame({"PassengerId":list(range(892, 1310)),"Survived":prediction}).to_csv(file_name,
                                                                                          index=False,header=True)

In [195]:
linear_prediction = opt_linear_clf.predict(test)
save_prediction(linear_prediction, 'linear_prediction.csv')

In [196]:
parameters = {'n_estimators': [100, 200, 500, 1000, 5000]}
rf_clf = RandomForestClassifier()

opt_rf_clf = GridSearchCV(rf_clf, parameters, cv=5, scoring='accuracy')
opt_rf_clf.fit(x_train, y_train)

print("Best params:", opt_rf_clf.best_params_)

Best params: {'n_estimators': 500}


In [197]:
prediction = opt_rf_clf.predict(x_val)
rf_result = accuracy_score(y_val, prediction)

print('Result accuracy:', rf_result)

Result accuracy: 0.9888888888888889


In [198]:
rf_prediction = opt_rf_clf.predict(test)    
save_prediction(rf_prediction, 'rf_prediction.csv')